In [1]:
# Clone the repository
!git clone https://github.com/habeshaethiopia/AlphaCare-Insurance-Solutions.git
%cd AlphaCare-Insurance-Solutions

# Change the working directory

# Install the necessary dependencies (if any)
# !pip install -r requirements.txt  # If you have a requirements.txt file

Cloning into 'AlphaCare-Insurance-Solutions'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 97 (delta 44), reused 50 (delta 18), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 1.62 MiB | 3.74 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/AlphaCare-Insurance-Solutions


In [2]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import sys
import os

# Ensure the src directory is in the Python path
sys.path.append(
    os.path.abspath(
        os.path.join(
            "..", "src" # Adjust the directory path as necessary
        )
    )
)
print(os.path.abspath(
        os.path.join(
            "..", "src" # Adjust the directory path as necessary
        )
    ))
from src.data_preparation import DataLoader
# Importing the necessary functions from data_processing.py

import pandas as pd


/content/src


In [3]:
# Load a sample dataset (replace with your own file path)
file_path = r"https://drive.google.com/file/d/1GOLY8weQTQxcv3zyTN35z7va-WZobgpU/view?usp=drive_link" # Change this to your actual CSV file path
DL=DataLoader(file_path)
df = DL.load_data_from_drive_zip(file_path)

# Show the first few rows of the dataset
df.head()


Detected encoding: ascii


/content/AlphaCare-Insurance-Solutions/src/data_preparation.py:68: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(text_file, sep='|', encoding=encoding)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [4]:
# Impute or remove missing data
df.fillna(df.median(numeric_only=True), inplace=True)  # For numeric columns
df.fillna("Unknown", inplace=True)  # For categorical columns


<ipython-input-4-8ffbd64a2eca>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)  # For categorical columns


In [5]:
# Example of creating new features
df['ClaimsToPremiumRatio'] = df['TotalClaims'] / (df['TotalPremium'] + 1e-5)  # Avoid division by zero
df['IsHighRisk'] = (df['ClaimsToPremiumRatio'] > 0.5).astype(int)


In [6]:
from sklearn.preprocessing import OneHotEncoder

# Perform one-hot encoding on categorical features
categorical_cols = ['Province', 'VehicleType', 'Gender']
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [7]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(columns=['TotalClaims', 'TotalPremium'])
y_claims = df_encoded['TotalClaims']
y_premium = df_encoded['TotalPremium']

# Split data into training and testing sets (80:20)
X_train, X_test, y_claims_train, y_claims_test = train_test_split(X, y_claims, test_size=0.2, random_state=42)
_, _, y_premium_train, y_premium_test = train_test_split(X, y_premium, test_size=0.2, random_state=42)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Build and evaluate Linear Regression model
for col in X_train.select_dtypes(include=['object']).columns:
    if pd.api.types.is_datetime64_any_dtype(X_train[col]):
        X_train[col] = pd.to_datetime(X_train[col]).apply(lambda date: date.toordinal())
        X_test[col] = pd.to_datetime(X_test[col]).apply(lambda date: date.toordinal())
    #For the rest of the string columns convert to numbers
    else:
        X_train[col], _ = pd.factorize(X_train[col])
        X_test[col], _ = pd.factorize(X_test[col], sort=True)
linear_model = LinearRegression()
linear_model.fit(X_train, y_claims_train)
y_claims_pred = linear_model.predict(X_test)

mse = mean_squared_error(y_claims_test, y_claims_pred)
r2 = r2_score(y_claims_test, y_claims_pred)
print("Linear Regression Claims - MSE:", mse, "R2:", r2)


Linear Regression Claims - MSE: 3326888.4193807803 R2: 0.3191516839377394


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Build and evaluate Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_claims_train)
y_claims_pred_rf = rf_model.predict(X_test)

mse_rf = mean_squared_error(y_claims_test, y_claims_pred_rf)
r2_rf = r2_score(y_claims_test, y_claims_pred_rf)
print("Random Forest Claims - MSE:", mse_rf, "R2:", r2_rf)


In [ ]:
import xgboost as xgb

# Build and evaluate XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_claims_train)
y_claims_pred_xgb = xgb_model.predict(X_test)

mse_xgb = mean_squared_error(y_claims_test, y_claims_pred_xgb)
r2_xgb = r2_score(y_claims_test, y_claims_pred_xgb)
print("XGBoost Claims - MSE:", mse_xgb, "R2:", r2_xgb)


In [ ]:
# Compare model performances
print("Model Comparison for Total Claims:")
print(f"Linear Regression - MSE: {mse}, R2: {r2}")
print(f"Random Forest - MSE: {mse_rf}, R2: {r2_rf}")
print(f"XGBoost - MSE: {mse_xgb}, R2: {r2_xgb}")


In [ ]:
import shap

# Explain model predictions using SHAP
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

# Visualize feature importance
shap.summary_plot(shap_values, X_test)


In [ ]:
!pip install lime
from lime.lime_tabular import LimeTabularExplainer

# Explain specific predictions using LIME
lime_explainer = LimeTabularExplainer(X_train.values, feature_names=X_train.columns, class_names=['TotalClaims'], mode='regression')
exp = lime_explainer.explain_instance(X_test.iloc[0].values, rf_model.predict)
exp.show_in_notebook()
